# Case Study 4

In this notebook we rank systems using metrics on GLUE benchmark, their efficiency rate and data about social bias.

In [1]:
import pandas as pd
import numpy as np
import sys

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

sys.path.append("..")

from votenrank import Leaderboard

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

loading region bounding boxes for computing carbon emissions region, this may take a moment...
 454/454... rate=436.31 Hz, eta=0:00:00, total=0:00:010
Done!


/home/data_sapiens/.local/lib/python3.8/site-packages/experiment_impact_tracker/data_interface.py:37: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option("display.max_colwidth", -1)


## Load data

We multiply all efficiency values by -1 because they have to be minimized and not maximized.

In [6]:
data = pd.read_csv("../tables/case_study_4/glue_hpc.csv").set_index("model_name").drop("t5-base")
ethics = pd.read_csv("../tables/case_study_4/social_bias_data.csv", index_col="model_name").drop("t5-base")
tracker = pd.read_csv("../tables/case_study_4/experiment_tracker_table.csv", index_col="model_name")

agg_data = data.join(ethics, on=data.index).join(-tracker, on=data.index).astype(float)
agg_data

,wnli.eval_accuracy,cola.eval_matthews_correlation,stsb.eval_pearson,stsb.eval_spearmanr,rte.eval_accuracy,mrpc.eval_accuracy,mrpc.eval_f1,sst2.eval_accuracy,qnli.eval_accuracy,qqp.eval_accuracy,qqp.eval_f1,mnli.eval_accuracy,mnli-mm.eval_accuracy,anti_crows_score,stereoset_intra_icat,stereoset_inter_icat,winogender_acc_difference,winobias_acc_difference,rte_gpu_hours,stsb_kg_carbon,wnli_total_power,stsb_total_power,mnli_total_power,qnli_kg_carbon,mnli-mm_exp_len_hours,qnli_exp_len_hours,qqp_exp_len_hours,rte_total_power,rte_kg_carbon,sst2_kg_carbon,qqp_kg_carbon,rte_exp_len_hours,qnli_gpu_hours,qqp_gpu_hours,mnli_exp_len_hours,mrpc_gpu_hours,wnli_gpu_hours,wnli_kg_carbon,stsb_exp_len_hours,mrpc_kg_carbon,mnli-mm_gpu_hours,mnli_kg_carbon,wnli_exp_len_hours,stsb_gpu_hours,sst2_gpu_hours,mnli_gpu_hours,qqp_total_power,cola_kg_carbon,cola_gpu_hours,cola_total_power,mrpc_total_power,sst2_total_power,sst2_exp_len_hours,mrpc_exp_len_hours,cola_exp_len_hours,mnli-mm_total_power,mnli-mm_kg_carbon,qnli_total_power
model_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
distilbert-base-uncased,54.6,53.9,87.1,86.8,62.9,84.7,89.6,90.9,89.2,90.4,87.1,82.2,82.3,0.4098,0.6998,0.7528,0.9917,0.9981,-0.009423,-0.001357,-0.001057,-0.005413,-1.069909,-0.062459,-4.480067,-0.616156,-2.227646,-0.004557,-0.001143,-0.010247,-0.161034,-0.012203,-0.452701,-1.095744,-4.265462,-0.006452,-0.000999,-0.000265,-0.016014,-0.000791,-1.441347,-0.268262,-0.003287,-0.010926,-0.096281,-1.447840,-0.642252,-0.001257,-0.011305,-0.005013,-0.003155,-0.040868,-0.150034,-0.009880,-0.019829,-1.059574,-0.265671,-0.249105
bert-base-uncased,42.5,59.0,89.7,89.3,69.7,85.8,90.0,92.9,91.5,91.2,88.2,84.4,84.9,0.4138,0.7279,0.6700,0.9875,0.9552,-0.017499,-0.002484,-0.001444,-0.009909,-2.095817,-0.126461,-8.408221,-1.312409,-4.697835,-0.008031,-0.002014,-0.018614,-0.329854,-0.022321,-0.861943,-2.067670,-8.857196,-0.012200,-0.001466,-0.000362,-0.028338,-0.001327,-2.742530,-0.525491,-0.004337,-0.020176,-0.178940,-2.741965,-1.315557,-0.002203,-0.020750,-0.008787,-0.005294,-0.074237,-0.283502,-0.018210,-0.036952,-1.959109,-0.491214,-0.504366
roberta-base,56.3,60.4,90.6,90.4,76.0,89.1,92.1,94.1,92.5,91.5,88.7,87.5,87.2,0.3773,0.6750,0.3995,0.9708,0.9609,-0.016651,-0.002488,-0.001763,-0.009921,-2.157483,-0.130972,-8.096763,-1.342347,-5.054267,-0.008468,-0.002123,-0.019406,-0.330371,-0.020526,-0.898159,-2.097117,-10.052896,-0.011822,-0.001870,-0.000442,-0.029009,-0.001486,-2.684118,-0.540953,-0.004431,-0.020887,-0.184717,-2.694672,-1.317620,-0.002337,-0.021341,-0.009321,-0.005925,-0.077397,-0.287918,-0.017837,-0.036028,-1.985482,-0.497827,-0.522354
distilroberta-base,51.5,56.1,86.9,86.7,64.5,84.0,88.7,91.8,90.9,90.9,87.8,84.0,84.3,0.3687,0.6833,0.6247,0.9958,0.9861,-0.008836,-0.001292,-0.000875,-0.005154,-0.732750,-0.061111,-1.757867,-0.558800,-1.494409,-0.004944,-0.001240,-0.010311,-0.136711,-0.012071,-0.470668,-1.116539,-1.956517,-0.006609,-0.001007,-0.000219,-0.016254,-0.000727,-1.424475,-0.183725,-0.002817,-0.011469,-0.101979,-1.427339,-0.545243,-0.001277,-0.012039,-0.005094,-0.002901,-0.041124,-0.159032,-0.010284,-0.020594,-0.718782,-0.180223,-0.243730
albert-base-v2,54.1,57.1,90.8,90.4,77.5,87.6,91.0,91.8,91.0,90.6,87.4,84.4,84.6,0.4410,0.6901,0.7770,0.9958,0.9924,-0.017268,-0.002204,-0.001224,-0.008790,-2.660270,-0.132740,-9.933616,-1.447277,-7.815161,-0.008678,-0.002176,-0.016236,-0.436818,-0.020173,-0.911492,-2.140980,-12.785114,-0.013097,-0.001538,-0.000307,-0.022631,-0.001735,-2.924104,-0.667019,-0.002903,-0.018657,-0.147321,-2.923310,-1.742160,-0.001577,-0.014013,-0.006288,-0.006921,-0.064754,-0.195145,-0.015761,-0.023249,-2.292798,-0.574881,-0.529408
deberta-base,56.3,62.9,91.1,90.8,72.6,89.8,92.7,94.5,92.8,91.6,88.7,87.9,87.9,0.5159,0.4248,0.4035,0.9917,0.9867,-0.030452,-0.003464,-0.001967,-0.013816,-2.923224,-0.238413,-17.840316,-2.713213,-7.512733,-0.017037,-0.004272,-0.027810,-0.524498,-0.035368,-1.681553,-3.381128,-10.030674,-0.017665,-0.003556,-0.000493,-0.0

## Ranking and winner selection without group weights

In [7]:
weights = {col: 0.5 for col in agg_data.columns if col.split(".")[0] in ["stsb", "mrpc", "qqp"]}
naive_lb = Leaderboard(agg_data, weights=weights)
naive_lb.rank_all()

,Borda,AM,GM,Copeland,Plurality,Minimax,Dowdall
Ranking position,,,,,,,
1,251.00: distilroberta-base,14.32: roberta-base,nan: distilbert-base-uncased,6.00: distilroberta-base,23.00: distilroberta-base,-0.00: distilroberta-base,35.24: distilroberta-base
2,247.00: distilbert-base-uncased,14.11: distilroberta-base,nan: bert-base-uncased,4.00: distilbert-base-uncased,18.00: distilbert-base-uncased,-31.00: distilbert-base-uncased,32.59: distilbert-base-uncased
3,167.00: bert-base-uncased,13.91: deberta-base,nan: roberta-base,2.00: bert-base-uncased,8.50: deberta-base,-42.00: albert-base-v2,17.02: deberta-base
4,158.50: albert-base-v2,13.87: albert-base-v2,nan: distilroberta-base,0.00: roberta-base,2.00: albert-base-v2,-43.00: deberta-base,16.90: albert-base-v2
5,144.00: roberta-base,13.86: distilbert-base-uncased,nan: albert-base-v2,-2.00: albert-base-v2,1.00: bert-base-uncased,-44.00: bert-base-uncased,15.37: roberta-base
6,107.00: gpt2,13.74: bert-base-uncased,nan: deberta-base,-4.00: gpt2,0.00: roberta-base,-45.00: roberta-base,15.31: bert-base-uncased
7,72.50: deberta-base,13.18: gpt2,nan: gpt2,-6.00: deberta-base,0.00: gpt2,-49.00: gpt2,12.34: gpt2


In [8]:
naive_lb.elect_all()

,method,winners
0,Borda,[distilroberta-base]
1,AM,[roberta-base]
2,GM,[]
3,Condorcet,[distilroberta-base]
4,Copeland,[distilroberta-base]
5,Baldwin,[distilroberta-base]
6,Plurality,[distilroberta-base]
7,Threshold,[distilroberta-base]
8,Minimax,[distilroberta-base]
9,Dowdall,[distilroberta-base]


## Weighted ranking: performance, efficiency and social bias have equal weights

In [9]:
task_groups = {
    "metrics": data.columns.tolist(),
    "efficiency": tracker.columns.tolist(),
    "ethics": ethics.columns.tolist()
}

group_weights = naive_lb.weights.copy()

for column in agg_data.columns:
    for group, tasks in task_groups.items():
        if column in tasks:
            group_weights.loc[column] /= len(tasks)

print("Group weights")
group_weights

Group weights


wnli.eval_accuracy                0.076923
cola.eval_matthews_correlation    0.076923
stsb.eval_pearson                 0.038462
stsb.eval_spearmanr               0.038462
rte.eval_accuracy                 0.076923
mrpc.eval_accuracy                0.038462
mrpc.eval_f1                      0.038462
sst2.eval_accuracy                0.076923
qnli.eval_accuracy                0.076923
qqp.eval_accuracy                 0.038462
qqp.eval_f1                       0.038462
mnli.eval_accuracy                0.076923
mnli-mm.eval_accuracy             0.076923
anti_crows_score                  0.200000
stereoset_intra_icat              0.200000
stereoset_inter_icat              0.200000
winogender_acc_difference         0.200000
winobias_acc_difference           0.200000
rte_gpu_hours                     0.025000
stsb_kg_carbon                    0.025000
wnli_total_power                  0.025000
stsb_total_power                  0.025000
mnli_total_power                  0.025000
qnli_kg_car

In [10]:
weighted_lb = Leaderboard(agg_data, weights=group_weights.to_dict())
weighted_lb.rank_all()

,Borda,AM,GM,Copeland,Plurality,Minimax,Dowdall
Ranking position,,,,,,,
1,10.25: distilbert-base-uncased,22.79: roberta-base,nan: distilbert-base-uncased,6.00: distilbert-base-uncased,0.78: deberta-base,-0.00: distilbert-base-uncased,1.34: distilbert-base-uncased
2,9.67: albert-base-v2,22.63: deberta-base,nan: bert-base-uncased,4.00: albert-base-v2,0.63: distilbert-base-uncased,-1.45: albert-base-v2,1.30: distilroberta-base
3,9.08: distilroberta-base,22.31: albert-base-v2,nan: roberta-base,2.00: distilroberta-base,0.58: distilroberta-base,-1.46: distilroberta-base,1.24: deberta-base
4,8.46: bert-base-uncased,21.97: distilroberta-base,nan: distilroberta-base,0.00: bert-base-uncased,0.28: albert-base-v2,-1.68: bert-base-uncased,1.18: albert-base-v2
5,7.02: deberta-base,21.91: bert-base-uncased,nan: albert-base-v2,-4.00: roberta-base,0.20: bert-base-uncased,-1.83: deberta-base,0.87: bert-base-uncased
6,6.84: roberta-base,21.73: distilbert-base-uncased,nan: deberta-base,-4.00: deberta-base,0.00: roberta-base,-2.00: roberta-base,0.82: roberta-base
7,5.72: gpt2,21.02: gpt2,nan: gpt2,-4.00: gpt2,0.00: gpt2,-2.06: gpt2,0.76: gpt2


In [11]:
weighted_lb.elect_all()

,method,winners
0,Borda,[distilbert-base-uncased]
1,AM,[roberta-base]
2,GM,[]
3,Condorcet,[distilbert-base-uncased]
4,Copeland,[distilbert-base-uncased]
5,Baldwin,[distilbert-base-uncased]
6,Plurality,[deberta-base]
7,Threshold,[albert-base-v2]
8,Minimax,[distilbert-base-uncased]
9,Dowdall,[distilbert-base-uncased]


## Ranking with respect to groups

### Ranking according to performance

In [13]:
Leaderboard(agg_data[task_groups["metrics"]], 
            weights=naive_lb.weights[task_groups["metrics"]].to_dict()).rank_all()

,Borda,AM,GM,Copeland,Plurality,Minimax,Dowdall
Ranking position,,,,,,,
1,56.50: deberta-base,82.73: deberta-base,81.56: deberta-base,6.00: deberta-base,7.50: deberta-base,-0.00: deberta-base,9.33: deberta-base
2,49.00: roberta-base,82.52: roberta-base,81.34: roberta-base,4.00: roberta-base,1.00: albert-base-v2,-7.50: roberta-base,5.67: roberta-base
3,32.50: albert-base-v2,80.94: albert-base-v2,79.65: albert-base-v2,2.00: bert-base-uncased,0.00: distilbert-base-uncased,-9.00: albert-base-v2,3.57: albert-base-v2
4,32.00: bert-base-uncased,79.20: bert-base-uncased,77.19: bert-base-uncased,0.00: albert-base-v2,0.00: bert-base-uncased,-10.00: distilbert-base-uncased,2.89: bert-base-uncased
5,17.00: distilroberta-base,78.56: distilroberta-base,77.05: distilroberta-base,-2.00: distilroberta-base,0.00: roberta-base,-10.00: bert-base-uncased,1.95: distilroberta-base
6,11.00: distilbert-base-uncased,77.89: distilbert-base-uncased,76.46: distilbert-base-uncased,-4.00: distilbert-base-uncased,0.00: distilroberta-base,-10.00: distilroberta-base,1.80: distilbert-base-uncased
7,8.00: gpt2,75.95: gpt2,73.56: gpt2,-6.00: gpt2,0.00: gpt2,-10.00: gpt2,1.66: gpt2


### Ranking according to efficiency

In [14]:
Leaderboard(agg_data[task_groups["efficiency"]], 
            weights=naive_lb.weights[task_groups["efficiency"]].to_dict()).rank_all()

,Borda,AM,GM,Copeland,Plurality,Minimax,Dowdall
Ranking position,,,,,,,
1,223.00: distilroberta-base,-0.34: distilroberta-base,nan: distilbert-base-uncased,6.00: distilroberta-base,23.00: distilroberta-base,-0.00: distilroberta-base,31.50: distilroberta-base
2,216.00: distilbert-base-uncased,-0.51: distilbert-base-uncased,nan: bert-base-uncased,4.00: distilbert-base-uncased,17.00: distilbert-base-uncased,-23.00: distilbert-base-uncased,28.33: distilbert-base-uncased
3,120.00: bert-base-uncased,-0.96: gpt2,nan: roberta-base,2.00: bert-base-uncased,0.00: bert-base-uncased,-40.00: bert-base-uncased,10.50: bert-base-uncased
4,103.00: albert-base-v2,-1.00: bert-base-uncased,nan: distilroberta-base,0.00: roberta-base,0.00: roberta-base,-40.00: roberta-base,10.09: albert-base-v2
5,91.00: roberta-base,-1.03: roberta-base,nan: albert-base-v2,-2.00: albert-base-v2,0.00: albert-base-v2,-40.00: albert-base-v2,8.88: roberta-base
6,84.00: gpt2,-1.26: albert-base-v2,nan: deberta-base,-4.00: gpt2,0.00: deberta-base,-40.00: deberta-base,8.62: gpt2
7,3.00: deberta-base,-1.64: deberta-base,nan: gpt2,-6.00: deberta-base,0.00: gpt2,-40.00: gpt2,5.80: deberta-base


### Ranking according to social bias

In [15]:
Leaderboard(agg_data[task_groups["ethics"]], 
            weights=naive_lb.weights[task_groups["ethics"]].to_dict()).rank_all()

,Borda,AM,GM,Copeland,Plurality,Minimax,Dowdall
Ranking position,,,,,,,
1,23.00: albert-base-v2,0.78: albert-base-v2,0.75: albert-base-v2,6.00: albert-base-v2,1.00: distilbert-base-uncased,-0.00: albert-base-v2,3.25: albert-base-v2
2,20.00: distilbert-base-uncased,0.77: distilbert-base-uncased,0.73: distilbert-base-uncased,4.00: distilbert-base-uncased,1.00: bert-base-uncased,-3.00: distilbert-base-uncased,2.45: distilbert-base-uncased
3,15.00: bert-base-uncased,0.75: bert-base-uncased,0.72: bert-base-uncased,0.00: bert-base-uncased,1.00: albert-base-v2,-3.00: gpt2,2.06: gpt2
4,15.00: gpt2,0.74: gpt2,0.71: gpt2,0.00: gpt2,1.00: deberta-base,-4.00: bert-base-uncased,1.92: bert-base-uncased
5,13.00: deberta-base,0.73: distilroberta-base,0.69: distilroberta-base,-2.00: distilroberta-base,0.00: roberta-base,-4.00: distilroberta-base,1.89: deberta-base
6,11.00: distilroberta-base,0.68: roberta-base,0.62: roberta-base,-2.00: deberta-base,0.00: distilroberta-base,-4.00: deberta-base,1.79: distilroberta-base
7,4.00: roberta-base,0.66: deberta-base,0.61: deberta-base,-6.00: roberta-base,0.00: gpt2,-5.00: roberta-base,0.82: roberta-base


### Two-step ranking with weights

In [12]:
group_weights = {
    "metrics": 0.4,
    "efficiency": 0.3,
    "ethics": 0.3
}

naive_lb.rank_all(task_groups=task_groups, group_weights=group_weights)

,Borda,AM,GM,Copeland,Plurality,Minimax,Dowdall
Ranking position,,,,,,,
1,4.30: albert-base-v2,32.90: roberta-base,nan: distilbert-base-uncased,4.00: bert-base-uncased,0.40: deberta-base,-0.40: albert-base-v2,0.51: albert-base-v2
2,3.40: distilbert-base-uncased,32.80: deberta-base,nan: bert-base-uncased,2.00: distilbert-base-uncased,0.30: distilroberta-base,-0.60: deberta-base,0.50: deberta-base
3,3.30: bert-base-uncased,32.23: albert-base-v2,nan: roberta-base,2.00: albert-base-v2,0.00: distilbert-base-uncased,-0.70: distilbert-base-uncased,0.43: distilroberta-base
4,3.00: deberta-base,31.61: bert-base-uncased,nan: distilroberta-base,0.00: distilroberta-base,0.00: bert-base-uncased,-0.70: bert-base-uncased,0.37: distilbert-base-uncased
5,2.90: distilroberta-base,31.54: distilroberta-base,nan: albert-base-v2,-2.00: roberta-base,0.00: roberta-base,-0.70: roberta-base,0.30: roberta-base
6,2.60: roberta-base,31.23: distilbert-base-uncased,nan: deberta-base,-2.00: deberta-base,0.00: albert-base-v2,-0.70: distilroberta-base,0.28: bert-base-uncased
7,1.20: gpt2,30.32: gpt2,nan: gpt2,-4.00: gpt2,0.00: gpt2,-0.70: gpt2,0.21: gpt2
